In [1]:
from sqlalchemy import create_engine

In [2]:
engine = create_engine(
    "sqlite:///../output/zoo.db", echo=True, connect_args={"check_same_thread": False}
)

In [3]:
from sqlalchemy import MetaData
from sqlalchemy import Table, Column, Integer, Numeric, String, ForeignKey, Boolean

metadata = MetaData()

In [4]:
cookies = Table(
    "cookies",
    metadata,
    Column("cookie_id", Integer(), primary_key=True),
    Column("cookie_name", String(50), index=True),
    Column("cookie_url", String(255)),
    Column("cookie_sku", String(55)),
    Column("quantity", Integer()),
    Column("unit_cost", Numeric(12, 2)),
)
from datetime import datetime
from sqlalchemy import DateTime

users = Table(
    "users",
    metadata,
    Column("user_id", Integer(), primary_key=True),
    Column("username", String(15), nullable=False, unique=True),
    Column("email_address", String(255), nullable=False),
    Column("phone", String(20), nullable=False),
    Column("password", String(25), nullable=False),
    Column("create_on", DateTime(), default=datetime.now),
    Column("update_on", DateTime(), default=datetime.now, onupdate=datetime.now),
)
from sqlalchemy import ForeignKey, Boolean

orders = Table(
    "orders",
    metadata,
    Column("order_id", Integer(), primary_key=True),
    Column("user_id", ForeignKey("users.user_id")),
    Column("shipped", Boolean(), default=False),
)
line_items = Table(
    "line_items",
    metadata,
    Column("line_id", Integer(), primary_key=True),
    Column("order_id", ForeignKey("orders.order_id")),
    Column("cookie_id", ForeignKey("cookies.cookie_id")),
    Column("quantity", Integer()),
    Column("extended_cost", Numeric(12, 2)),
)
metadata.create_all(engine)

2025-05-06 14:28:13,914 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-06 14:28:13,915 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("cookies")
2025-05-06 14:28:13,916 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-06 14:28:13,917 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("cookies")
2025-05-06 14:28:13,918 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-06 14:28:13,918 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2025-05-06 14:28:13,919 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-06 14:28:13,920 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2025-05-06 14:28:13,921 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-06 14:28:13,922 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("orders")
2025-05-06 14:28:13,922 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-05-06 14:28:13,923 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("orders")
2025-05-06 14:28:13,923 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-

In [5]:
ins = cookies.insert().values(
    cookie_name="chocolate chip",
    cookie_url="http://some.aweso.me/cookie/recipe.html",
    cookie_sku="CC01",
    quantity="12",
    unit_cost="0.50",
)
print(str(ins))

INSERT INTO cookies (cookie_name, cookie_url, cookie_sku, quantity, unit_cost) VALUES (:cookie_name, :cookie_url, :cookie_sku, :quantity, :unit_cost)


In [6]:
from sqlalchemy import insert

In [7]:
ins = insert(cookies).values(
    cookie_name="chocolate chip",
    cookie_url="http://some.aweso.me/cookie/recipe.html",
    cookie_sku="CC01",
    quantity="12",
    unit_cost="0.50",
)

In [8]:
print(str(ins))

INSERT INTO cookies (cookie_name, cookie_url, cookie_sku, quantity, unit_cost) VALUES (:cookie_name, :cookie_url, :cookie_sku, :quantity, :unit_cost)


In [9]:
cnn = engine.connect()
cnn.execute(ins)
cnn.commit()

2025-05-06 14:28:15,304 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-06 14:28:15,305 INFO sqlalchemy.engine.Engine INSERT INTO cookies (cookie_name, cookie_url, cookie_sku, quantity, unit_cost) VALUES (?, ?, ?, ?, ?)
2025-05-06 14:28:15,306 INFO sqlalchemy.engine.Engine [generated in 0.00196s] ('chocolate chip', 'http://some.aweso.me/cookie/recipe.html', 'CC01', '12', 0.5)
2025-05-06 14:28:15,307 INFO sqlalchemy.engine.Engine COMMIT


In [10]:
inventory_list = [
    {
        "cookie_name": "peanut butter",
        "cookie_recipe_url": "http://some.aweso.me/cookie/peanut.html",
        "cookie_sku": "PB01",
        "quantity": "24",
        "unit_cost": "0.25",
    },
    {
        "cookie_name": "oatmeal raisin",
        "cookie_recipe_url": "http://some.okay.me/cookie/raisin.html",
        "cookie_sku": "EWW01",
        "quantity": "100",
        "unit_cost": "1.00",
    },
]
result = cnn.execute(ins, inventory_list)
cnn.commit()

2025-05-06 14:29:57,733 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-06 14:29:57,734 INFO sqlalchemy.engine.Engine INSERT INTO cookies (cookie_name, cookie_url, cookie_sku, quantity, unit_cost) VALUES (?, ?, ?, ?, ?)
2025-05-06 14:29:57,735 INFO sqlalchemy.engine.Engine [generated in 0.00226s] [('peanut butter', 'http://some.aweso.me/cookie/recipe.html', 'PB01', '24', 0.25), ('oatmeal raisin', 'http://some.aweso.me/cookie/recipe.html', 'EWW01', '100', 1.0)]
2025-05-06 14:29:57,737 INFO sqlalchemy.engine.Engine COMMIT


In [11]:
from sqlalchemy.sql import select

s = select(cookies)
rp = cnn.execute(s)
result = rp.fetchall()
print(result)

2025-05-06 14:30:52,707 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-06 14:30:52,710 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id, cookies.cookie_name, cookies.cookie_url, cookies.cookie_sku, cookies.quantity, cookies.unit_cost 
FROM cookies
2025-05-06 14:30:52,711 INFO sqlalchemy.engine.Engine [generated in 0.00391s] ()
[(1, 'chocolate chip', 'http://some.aweso.me/cookie/recipe.html', 'CC01', 12, Decimal('0.50')), (2, 'peanut butter', 'http://some.aweso.me/cookie/recipe.html', 'PB01', 24, Decimal('0.25')), (3, 'oatmeal raisin', 'http://some.aweso.me/cookie/recipe.html', 'EWW01', 100, Decimal('1.00'))]


In [12]:
str(s)

'SELECT cookies.cookie_id, cookies.cookie_name, cookies.cookie_url, cookies.cookie_sku, cookies.quantity, cookies.unit_cost \nFROM cookies'

In [21]:
s = cookies.select()
rp = cnn.execute(s)
# print(rp.fetchall())
result = rp.fetchall()
result

2025-05-06 14:34:17,188 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id, cookies.cookie_name, cookies.cookie_url, cookies.cookie_sku, cookies.quantity, cookies.unit_cost 
FROM cookies
2025-05-06 14:34:17,190 INFO sqlalchemy.engine.Engine [cached since 204.5s ago] ()


[(1, 'chocolate chip', 'http://some.aweso.me/cookie/recipe.html', 'CC01', 12, Decimal('0.50')),
 (2, 'peanut butter', 'http://some.aweso.me/cookie/recipe.html', 'PB01', 24, Decimal('0.25')),
 (3, 'oatmeal raisin', 'http://some.aweso.me/cookie/recipe.html', 'EWW01', 100, Decimal('1.00'))]

In [23]:
firstrow = result[0]
firstrow

(1, 'chocolate chip', 'http://some.aweso.me/cookie/recipe.html', 'CC01', 12, Decimal('0.50'))

In [24]:
firstrow[1]

'chocolate chip'

In [25]:
firstrow.cookie_name

'chocolate chip'

In [27]:
rp = cnn.execute(s)
for re in rp:
    print(re.cookie_name)

2025-05-06 14:35:57,344 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id, cookies.cookie_name, cookies.cookie_url, cookies.cookie_sku, cookies.quantity, cookies.unit_cost 
FROM cookies
2025-05-06 14:35:57,345 INFO sqlalchemy.engine.Engine [cached since 304.6s ago] ()
chocolate chip
peanut butter
oatmeal raisin


In [31]:
ins = insert(cookies).values(
    cookie_name="dark chocolate chip",
    cookie_url="http://some.aweso.me/cookie/recipe_dark.html",
    cookie_sku="CC02",
    quantity="1",
    unit_cost="0.75",
)
result = cnn.execute(
    ins,
)
cnn.commit()
result.inserted_primary_key

2025-05-06 14:41:53,483 INFO sqlalchemy.engine.Engine INSERT INTO cookies (cookie_name, cookie_url, cookie_sku, quantity, unit_cost) VALUES (?, ?, ?, ?, ?)
2025-05-06 14:41:53,485 INFO sqlalchemy.engine.Engine [cached since 818.2s ago] ('dark chocolate chip', 'http://some.aweso.me/cookie/recipe_dark.html', 'CC02', '1', 0.75)
2025-05-06 14:41:53,487 INFO sqlalchemy.engine.Engine COMMIT


(4,)

In [32]:
s = select(cookies.c.cookie_name, cookies.c.quantity)
rp = cnn.execute(s)

2025-05-06 14:43:20,466 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-06 14:43:20,468 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_name, cookies.quantity 
FROM cookies
2025-05-06 14:43:20,469 INFO sqlalchemy.engine.Engine [generated in 0.00278s] ()


In [33]:
print(rp.keys())

RMKeyView(['cookie_name', 'quantity'])


In [34]:
result = rp.first()
result

('chocolate chip', 12)

In [35]:
s = select(cookies.c.cookie_name, cookies.c.quantity)
s = s.order_by(cookies.c.quantity)
rp = cnn.execute(s)
for row in rp:
    print(f"{row.quantity}--{row.cookie_name}")

2025-05-06 14:45:16,533 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_name, cookies.quantity 
FROM cookies ORDER BY cookies.quantity
2025-05-06 14:45:16,535 INFO sqlalchemy.engine.Engine [generated in 0.00214s] ()
1--dark chocolate chip
12--chocolate chip
24--peanut butter
100--oatmeal raisin


In [36]:
from sqlalchemy import desc

s = select(cookies.c.cookie_name, cookies.c.quantity)
s = s.order_by(desc(cookies.c.quantity))
rep = cnn.execute(s)
for row in rep:
    print(row)

2025-05-06 14:47:06,462 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_name, cookies.quantity 
FROM cookies ORDER BY cookies.quantity DESC
2025-05-06 14:47:06,464 INFO sqlalchemy.engine.Engine [generated in 0.00149s] ()
('oatmeal raisin', 100)
('peanut butter', 24)
('chocolate chip', 12)
('dark chocolate chip', 1)


In [37]:
s = select(cookies.c.cookie_name, cookies.c.quantity)
s = s.order_by(cookies.c.quantity)
s = s.limit(2)
rp = cnn.execute(s)
print([res.cookie_name for res in rp])

2025-05-06 14:48:42,282 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_name, cookies.quantity 
FROM cookies ORDER BY cookies.quantity
 LIMIT ? OFFSET ?
2025-05-06 14:48:42,284 INFO sqlalchemy.engine.Engine [generated in 0.00248s] (2, 0)
['dark chocolate chip', 'chocolate chip']


In [38]:
from sqlalchemy.sql import func

s = select(func.sum(cookies.c.quantity))
rp = cnn.execute(s)
print(rp.scalar())

2025-05-06 14:49:47,108 INFO sqlalchemy.engine.Engine SELECT sum(cookies.quantity) AS sum_1 
FROM cookies
2025-05-06 14:49:47,110 INFO sqlalchemy.engine.Engine [generated in 0.00160s] ()
137


In [39]:
s = select(func.count(cookies.c.cookie_name))
rp = cnn.execute(s)
res = rp.first()

2025-05-06 14:50:40,912 INFO sqlalchemy.engine.Engine SELECT count(cookies.cookie_name) AS count_1 
FROM cookies
2025-05-06 14:50:40,913 INFO sqlalchemy.engine.Engine [generated in 0.00173s] ()


In [48]:
s = select(cookies).where(cookies.c.cookie_name == "chocolate chip")
rp = cnn.execute(s)
res = rp.first()
print(res)

2025-05-06 14:53:11,170 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id, cookies.cookie_name, cookies.cookie_url, cookies.cookie_sku, cookies.quantity, cookies.unit_cost 
FROM cookies 
WHERE cookies.cookie_name = ?
2025-05-06 14:53:11,172 INFO sqlalchemy.engine.Engine [generated in 0.00208s] ('chocolate chip',)
(1, 'chocolate chip', 'http://some.aweso.me/cookie/recipe.html', 'CC01', 12, Decimal('0.50'))


In [49]:
s = select(cookies.c.cookie_name, "sku-" + cookies.c.cookie_sku)
for row in cnn.execute(s):
    print(row)

2025-05-06 14:56:05,338 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_name, ? || cookies.cookie_sku AS anon_1 
FROM cookies
2025-05-06 14:56:05,340 INFO sqlalchemy.engine.Engine [generated in 0.00172s] ('sku-',)
('chocolate chip', 'sku-CC01')
('peanut butter', 'sku-PB01')
('oatmeal raisin', 'sku-EWW01')
('dark chocolate chip', 'sku-CC02')


In [51]:
from sqlalchemy import cast

s = select(
    cookies.c.cookie_name,
    cast((cookies.c.quantity * cookies.c.unit_cost), Numeric(12, 2)).label("inv_cost"),
)
for row in cnn.execute(s):
    print(f"{row.cookie_name}--{row.inv_cost}")

2025-05-06 14:58:50,583 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_name, CAST(cookies.quantity * cookies.unit_cost AS NUMERIC(12, 2)) AS inv_cost 
FROM cookies
2025-05-06 14:58:50,584 INFO sqlalchemy.engine.Engine [cached since 37.03s ago] ()
chocolate chip--6.00
peanut butter--6.00
oatmeal raisin--100.00
dark chocolate chip--0.75


In [52]:
from sqlalchemy import update

u = update(cookies).where(cookies.c.cookie_name == "chocolate chip")
u = u.values(quantity=(cookies.c.quantity + 120))
result = cnn.execute(s).first()
cnn.execute(u)

2025-05-06 15:00:59,777 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_name, CAST(cookies.quantity * cookies.unit_cost AS NUMERIC(12, 2)) AS inv_cost 
FROM cookies
2025-05-06 15:00:59,779 INFO sqlalchemy.engine.Engine [cached since 166.2s ago] ()
2025-05-06 15:00:59,781 INFO sqlalchemy.engine.Engine UPDATE cookies SET quantity=(cookies.quantity + ?) WHERE cookies.cookie_name = ?
2025-05-06 15:00:59,782 INFO sqlalchemy.engine.Engine [generated in 0.00141s] (120, 'chocolate chip')


In [53]:
s = select(cookies).where(cookies.c.cookie_name == "chocolate chip")
result = cnn.execute(s).first()

2025-05-06 15:01:52,992 INFO sqlalchemy.engine.Engine SELECT cookies.cookie_id, cookies.cookie_name, cookies.cookie_url, cookies.cookie_sku, cookies.quantity, cookies.unit_cost 
FROM cookies 
WHERE cookies.cookie_name = ?
2025-05-06 15:01:52,995 INFO sqlalchemy.engine.Engine [cached since 521.8s ago] ('chocolate chip',)


In [57]:
[n for n in dir(result) if not n.startswith("_")]

['count', 'index', 't', 'tuple']

In [58]:
result.t

/tmp/ipykernel_77739/3161211357.py:1: SADeprecationWarning: The Row.t attribute is deprecated in favor of Row._t; all Row methods and library-level attributes are intended to be underscored to avoid name conflicts.  Please use Row._t. (deprecated since: 2.0.19)
  result.t


(1, 'chocolate chip', 'http://some.aweso.me/cookie/recipe.html', 'CC01', 132, Decimal('0.50'))

In [59]:
customer_list = [
    {
        "username": "cookiemon",
        "email_address": "mon@cookie.com",
        "phone": "111-111-1111",
        "password": "password",
    },
    {
        "username": "cakeeater",
        "email_address": "cakeeater@cake.com",
        "phone": "222-222-2222",
        "password": "password",
    },
    {
        "username": "pieguy",
        "email_address": "guy@pie.com",
        "phone": "333-333-3333",
        "password": "password",
    },
]
ins = users.insert()
cnn.execute(ins, customer_list)
cnn.commit()

2025-05-06 15:07:15,668 INFO sqlalchemy.engine.Engine INSERT INTO users (username, email_address, phone, password, create_on, update_on) VALUES (?, ?, ?, ?, ?, ?)
2025-05-06 15:07:15,670 INFO sqlalchemy.engine.Engine [generated in 0.00214s] [('cookiemon', 'mon@cookie.com', '111-111-1111', 'password', '2025-05-06 15:07:15.668808', '2025-05-06 15:07:15.668819'), ('cakeeater', 'cakeeater@cake.com', '222-222-2222', 'password', '2025-05-06 15:07:15.668822', '2025-05-06 15:07:15.668823'), ('pieguy', 'guy@pie.com', '333-333-3333', 'password', '2025-05-06 15:07:15.668825', '2025-05-06 15:07:15.668827')]
2025-05-06 15:07:15,672 INFO sqlalchemy.engine.Engine COMMIT


In [60]:
ins = insert(orders).values(user_id=1, order_id=1)
cnn.execute(ins)
ins = insert(line_items)
order_items = [
    {"order_id": 1, "cookie_id": 1, "quantity": 2, "extended_cost": 1.00},
    {"order_id": 1, "cookie_id": 3, "quantity": 12, "extended_cost": 3.00},
]
cnn.execute(ins, order_items)
cnn.commit()

2025-05-06 15:08:48,130 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-06 15:08:48,131 INFO sqlalchemy.engine.Engine INSERT INTO orders (order_id, user_id, shipped) VALUES (?, ?, ?)
2025-05-06 15:08:48,132 INFO sqlalchemy.engine.Engine [generated in 0.00237s] (1, 1, 0)
2025-05-06 15:08:48,134 INFO sqlalchemy.engine.Engine INSERT INTO line_items (order_id, cookie_id, quantity, extended_cost) VALUES (?, ?, ?, ?)
2025-05-06 15:08:48,135 INFO sqlalchemy.engine.Engine [generated in 0.00066s] [(1, 1, 2, 1.0), (1, 3, 12, 3.0)]
2025-05-06 15:08:48,137 INFO sqlalchemy.engine.Engine COMMIT


In [61]:
ins = insert(orders).values(user_id=2, order_id=2)
cnn.execute(ins)
cnn.commit()

2025-05-06 15:09:45,105 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-06 15:09:45,108 INFO sqlalchemy.engine.Engine INSERT INTO orders (order_id, user_id, shipped) VALUES (?, ?, ?)
2025-05-06 15:09:45,109 INFO sqlalchemy.engine.Engine [cached since 56.98s ago] (2, 2, 0)
2025-05-06 15:09:45,110 INFO sqlalchemy.engine.Engine COMMIT


In [62]:
ins = insert(line_items)
order_items = [
    {"order_id": 2, "cookie_id": 1, "quantity": 24, "extended_cost": 12.00},
    {"order_id": 2, "cookie_id": 4, "quantity": 6, "extended_cost": 6.00},
]
cnn.execute(ins, order_items)
cnn.commit()

2025-05-06 15:10:18,494 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-05-06 15:10:18,496 INFO sqlalchemy.engine.Engine INSERT INTO line_items (order_id, cookie_id, quantity, extended_cost) VALUES (?, ?, ?, ?)
2025-05-06 15:10:18,497 INFO sqlalchemy.engine.Engine [cached since 90.36s ago] [(2, 1, 24, 12.0), (2, 4, 6, 6.0)]
2025-05-06 15:10:18,499 INFO sqlalchemy.engine.Engine COMMIT


In [70]:
cols = [
    orders.c.order_id,
    users.c.username,
    users.c.phone,
    cookies.c.cookie_name,
    line_items.c.quantity,
    line_items.c.extended_cost,
]
cookiemon_orders = select(
    orders.c.order_id,
    users.c["username", "phone"],
    cookies.c.cookie_name,
    line_items.c["quantity", "extended_cost"],
)
cookiemon_orders = cookiemon_orders.select_from(
    orders.join(users).join(line_items).join(cookies)
).where(users.c.username == "cookiemon")
result = cnn.execute(cookiemon_orders).fetchall()
for row in result:
    print(row)

2025-05-06 15:26:51,087 INFO sqlalchemy.engine.Engine SELECT orders.order_id, users.username, users.phone, cookies.cookie_name, line_items.quantity, line_items.extended_cost 
FROM orders JOIN users ON users.user_id = orders.user_id JOIN line_items ON orders.order_id = line_items.order_id JOIN cookies ON cookies.cookie_id = line_items.cookie_id 
WHERE users.username = ?
2025-05-06 15:26:51,089 INFO sqlalchemy.engine.Engine [generated in 0.00210s] ('cookiemon',)
(1, 'cookiemon', '111-111-1111', 'chocolate chip', 2, Decimal('1.00'))
(1, 'cookiemon', '111-111-1111', 'oatmeal raisin', 12, Decimal('3.00'))


In [73]:
allorders = select(users.c.username, func.count(orders.c.order_id))
allorders = allorders.select_from(users.outerjoin(orders))
allorders = allorders.group_by(users.c.username)
res = cnn.execute(allorders).fetchall()
for row in res:
    print(row)

2025-05-06 15:49:50,402 INFO sqlalchemy.engine.Engine SELECT users.username, count(orders.order_id) AS count_1 
FROM users LEFT OUTER JOIN orders ON users.user_id = orders.user_id GROUP BY users.username
2025-05-06 15:49:50,404 INFO sqlalchemy.engine.Engine [cached since 16.89s ago] ()
('cakeeater', 1)
('cookiemon', 1)
('pieguy', 0)
